In [28]:
import tensorflow_datasets as tfds
(train, test), info = tfds.load(
    'higgs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files = True,
    with_info=True
)

valid = train.take(1000)

In [29]:
import tensorflow as tf

def extract_column(features):
    return features["class_label"]  # Change the index to extract the desired feature

def extract(features):
    return [value for key, value in features.items() if key != "class_label"]

def recon(data):
    return (data.map(extract_column), data.map(extract))


In [34]:
import numpy as np

def reconfig(dataset):
    xdata = []
    ydata = []

    for n in dataset:
        xdata.append(list(n.values())[0])
        ydata.append(list(n.values())[1:])
    
    xdata = np.array(xdata)
    ydata = np.array(ydata)
    return (xdata, ydata)

In [35]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the input shape (nvis)
nvis = 28

# Build the model
model = keras.Sequential([
    layers.Input(shape=(nvis,)),  # Input layer
    layers.Dense(300, activation='tanh', kernel_initializer=keras.initializers.RandomNormal(stddev=0.1), name='h0'),
    layers.Dense(300, activation='tanh', kernel_initializer=keras.initializers.RandomNormal(stddev=0.05), name='h1'),
    layers.Dense(300, activation='tanh', kernel_initializer=keras.initializers.RandomNormal(stddev=0.05), name='h2'),
    layers.Dense(300, activation='tanh', kernel_initializer=keras.initializers.RandomNormal(stddev=0.05), name='h3'),  # Hidden layer
    layers.Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.RandomNormal(stddev=0.001), name='y')  # Output layer
])

In [36]:
# Adjusting the data
train = train.take(500000)
test = test.take(100000)

xtrain, ytrain = reconfig(train)
xtest, ytest = reconfig(test)
xval, yval = reconfig(valid)

2024-10-28 05:34:41.132629: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [37]:
valData = tf.data.Dataset.from_tensor_slices((yval, xval))
trainData = tf.data.Dataset.from_tensor_slices((ytrain, xtrain))
testData = tf.data.Dataset.from_tensor_slices((ytest, xtest))

In [45]:
from tensorflow.keras.callbacks import Callback

class MomentumAdjustor(Callback):
    def __init__(self, start=0, saturate=500, final_momentum=0.99):
        super(MomentumAdjustor, self).__init__()
        self.start = start
        self.saturate = saturate
        self.final_momentum = final_momentum

    def on_epoch_begin(self, epoch, logs=None):
        if epoch >= self.start:
            # Calculate the new momentum value
            momentum = (self.final_momentum - 0) * (epoch - self.start) / self.saturate
            momentum = min(momentum, self.final_momentum)  # Clamp to final_momentum
            # Update the momentum in the optimizer
            self.model.optimizer.momentum = momentum
            print(f'\nEpoch {epoch + 1}: Adjusted momentum to {momentum:.4f}')

momentum_adjustor = MomentumAdjustor(start=0.9, saturate=200, final_momentum=0.99)

In [46]:
import datetime

batch_size = 100

train_dataset = trainData.batch(batch_size)
valid_dataset = valData.batch(batch_size)
test_dataset = testData.batch(batch_size)

In [51]:
initial_learning_rate = 0.05
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=1.0000002,
    staircase=True)

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, weight_decay=0.00001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['auc', 'accuracy'])

In [52]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Set up callbacks for monitoring and learning rate adjustment
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.0000002, patience=5, min_lr=0.000001)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Train the model
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=500,  # Adjust based on your max epochs criteria
    callbacks=[early_stopping, tensorboard_callback, momentum_adjustor]
)


Epoch 1/500


5000/5000 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.7442 - auc: 0.8242 - loss: 0.5119 - val_accuracy: 0.7301 - val_auc: 0.8124 - val_loss: 0.5289

Epoch 2: Adjusted momentum to 0.0005
Epoch 2/500
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7459 - auc: 0.8258 - loss: 0.5099 - val_accuracy: 0.7279 - val_auc: 0.8121 - val_loss: 0.5284

Epoch 3: Adjusted momentum to 0.0054
Epoch 3/500
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7473 - auc: 0.8278 - loss: 0.5074 - val_accuracy: 0.7301 - val_auc: 0.8116 - val_loss: 0.5304

Epoch 4: Adjusted momentum to 0.0104
Epoch 4/500
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7483 - auc: 0.8290 - loss: 0.5058 - val_accuracy: 0.7340 - val_auc: 0.8151 - val_loss: 0.5257

Epoch 5: Adjusted momentum to 0.0153
Epoch 5/500
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7501 - auc: 0.8308 - loss: 0.5035 - val_accuracy: 0.7322 - val_auc: 0.8137 - val_loss: 0.5315

Epoch 6: Adjusted momentum to 0.0203
Ep